This Juypter notebook shows you how to look for bookings

In [4]:
import json
import requests
from datetime import datetime, date
from typing import Optional, Dict, Any
with open("credentials.cfg") as data_file:
	data = json.load(data_file)

authUrl = data["urls"]["authUrl"]
baseUrl = data["urls"]["baseUrl"]
client_id = data["credentials"]["client_id"]
client_secret = data["credentials"]["client_secret"]
username = data["credentials"]["username"]
password = data["credentials"]["password"]

In [2]:
def getAccessToken():
	# Returns access_token
	#
	# authUrl:
	#	https://usoauth.plutora.com/
	# client_id & client_secret
	#	Values generated by Plutora platform, see Customizations>API
	# username & password
	#	User credentials used to access the Plutora platform
	#
	# If successful, Returns: "" or JSON-formatted response
	# On failure, Returns: response text
	#
	headers = {}
	headers['content-type'] = "application/x-www-form-urlencoded"
	headers['cache-control'] = "no-cache"
	url = authUrl + "oauth/token"
	payload  = "client_id=" + client_id + '&'
	payload += "client_secret=" + client_secret + '&'
	payload += "grant_type=" + 'password&'
	payload += "username=" + username.replace('@','%40') + '&'
	payload += "password=" + password + '&'
	response = requests.request("POST", url, data=payload, headers=headers)
	if not response.ok:
		print ("Access token: ")
		print(response.text)
		exit()
	access_token = response.json()['access_token']
	return access_token
	# ------End of def getAccessToken()-----------

In [6]:
def get_bookings(bearer_token: str, date_time: str = "2025-06-01T09:00:00") -> Dict[Any, Any]:
    """
    Retrieves bookings from Plutora API for a specific date/time.
    
    Args:
        date_time (str): The date and time to filter bookings (ISO format)
                        Default: "2025-06-01T09:00:00"
    
    Returns:
        Dict: JSON response from the API containing booking data
    """
    
    # Construct the filter query
    filter_query = f"`startDate` <= `{date_time}` and `endDate` >= `{date_time}`"
    
    # Construct the full URL
    url = f"https://ukapi.plutora.com/Bookings"
    
    # Set up headers
    headers = {
        'accept': 'application/json',
        'Plutora-Info': 'script.name=swagger',
        'Authorization': f'bearer {bearer_token}'
    }
    params = {
        'filter': filter_query
    }
    # Make the GET request
    response = requests.get(url, headers=headers, params=params)
    
    # Check if request was successful
    if response.ok:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        response.raise_for_status()

In [7]:
access_token=getAccessToken()
def get_current_datetime_string():
    """Get current datetime in the format expected by get_bookings"""
    return datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

bookings = get_bookings(access_token,get_current_datetime_string())

In [8]:
bookings

{'resultSet': [{'id': '09e089c5-c0f2-ef11-abc0-06b344cd22ba',
   'environmentId': 'c2c94a3c-3df0-ef11-abc0-06b344cd22ba',
   'isEnvironmentGroup': False,
   'releaseId': 'c6dddd6d-45f0-ef11-abc0-06b344cd22ba',
   'phaseId': 'd8dddd6d-45f0-ef11-abc0-06b344cd22ba',
   'tebrId': 'e3df89c5-c0f2-ef11-abc0-06b344cd22ba',
   'startDate': '2025-06-29T23:00:00',
   'endDate': '2025-07-06T23:00:00',
   'status': 'Pending',
   'state': 'Available',
   'statusUpdaterId': None,
   'timeStamp': None},
  {'id': '85e089c5-c0f2-ef11-abc0-06b344cd22ba',
   'environmentId': '534b8a42-3df0-ef11-abc0-06b344cd22ba',
   'isEnvironmentGroup': False,
   'releaseId': '00b336a6-66f0-ef11-abc0-06b344cd22ba',
   'phaseId': '13b336a6-66f0-ef11-abc0-06b344cd22ba',
   'tebrId': '5fe089c5-c0f2-ef11-abc0-06b344cd22ba',
   'startDate': '2025-06-29T23:00:00',
   'endDate': '2025-07-06T23:00:00',
   'status': 'Pending',
   'state': 'Available',
   'statusUpdaterId': None,
   'timeStamp': None},
  {'id': 'd6e4becb-c0f2-ef1